In [1]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import logging
from PhocusWire.PhocusWireSource import PhocuswireScraper
from Skift.SkiftSource import SkiftScraper
from database_utils.database_methods import Database

In [3]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [11]:
if __name__ == "__main__":
    sqlite_db = Database(DB_DIR = r"C:/Users/HP/Documents/repos/news_ingestion_data_pipeline/data", DB_FILE = "articles.db")
    latest_timestamp, is_incremental  = sqlite_db.get_latest_news_time()
    if is_incremental:
        logger.info("Initiating increment load...")
        logger.info(f"Latest record time stamp present in database : {latest_timestamp}")
    else:
        logger.info("Latest record timestamp not found in database.")
        logger.info("Initiating full load...")
    extracted_articles = []
    
    Skriftscraper = SkiftScraper()
    
    skift_articles = Skriftscraper.extract_articles(latest_timestamp)
    extracted_articles.extend(skift_articles)
    Phocuswirescraper = PhocuswireScraper()
    phocuswire_articles = Phocuswirescraper.extract_articles(latest_timestamp)
    extracted_articles.extend(phocuswire_articles)

    logger.info(f"Total articles extracted : {len(extracted_articles)}")

    sqlite_db.upsert_articles(extracted_articles)

  


INFO:__main__:Latest record timestamp not found in database.
INFO:__main__:Initiating full load...
INFO:Skift.SkiftSource:Sleeping for 2.2 seconds before next page request.
INFO:Skift.SkiftSource:Author not available for article id 2b9bf83d0e4a3475040dbd8ef9d38144 and article headline Unlocking Hotel Profits: How a Property’s DNA Impacts GDS Bookings.
INFO:Skift.SkiftSource:Author not available for article id 6c05e4514a7cb005e1b4d942ddecb3f1 and article headline Abu Dhabi Gets 10-Year License for Sphere Venues in MENA Region.
INFO:Skift.SkiftSource:Sleeping for 2.5 seconds before next page request.
INFO:Skift.SkiftSource:Sleeping for 2.9 seconds before next page request.
INFO:Skift.SkiftSource:Sleeping for 2.1 seconds before next page request.
INFO:Skift.SkiftSource:Author not available for article id c25d99908f431b681d7124df78b9d72c and article headline Built for Play: How Qiddiya City Creates a New Destination Paradigm.
INFO:Skift.SkiftSource:Author not available for article id b1f0c

In [12]:
sqlite_db.query_topn_articles()

('b6180012cdfcaab01451bded2196d26c', 'https://skift.com/2025/08/01/from-concur-to-spotnana-steve-singh-on-how-ai-could-fix-corporate-travel/', 'From Concur to Spotnana: Steve Singh on How AI Could Fix Corporate Travel', "Sean O'Neill", '2025-08-01T17:19:21', 'Skift', '2025-08-03 06:37:48')
('d030436466546bcf23aa4befbf4d08b6', 'https://skift.com/2025/08/01/delta-says-it-will-not-use-ai-to-target-customers/', 'Delta Responds to AI-Pricing Backlash: No ‘Individualized Prices Based on Personal Data’', 'Meghna Maharishi', '2025-08-01T15:05:14', 'Skift', '2025-08-03 06:37:48')
('8db7ffa1ba14adc8dd8348e7f5d1923d', 'https://skift.com/2025/08/01/u-s-dollar-slide-hurts-accor-minor-and-melia/', 'U.S. Dollar Slide Hurts Accor, Minor, and Meliá', 'Luke Martin', '2025-08-01T13:53:29', 'Skift', '2025-08-03 06:37:48')
('3fa30e2c6eee18976f224053633c1a27', 'https://skift.com/2025/08/01/winners-losers-and-lots-of-premium-seats-europes-airline-scorecard/', 'Winners, Losers, and Lots of Premium Seats: Euro